# Script thats scrapes average flight fare between SFO and destination cities. Used as a feature to estimate size/length of a flight

In [128]:
from bs4 import BeautifulSoup
import requests
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pickle
import statistics
import pandas as pd

In [111]:
# Scrape average fare cost data from faredetective.com for each unique destination airport in flight data
def get_price_info():
    chromedriver = "/Users/dcotes/documents/chromedriver/chromedriver" 
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    
    dest_list = pd.read_pickle('dest_list.pkl')
    dest_dict = {}
    
    for dest in dest_list:
        driver.get("https://www.faredetective.com/")
        
        from_air = driver.find_element_by_id("fromAir")
        from_air.send_keys("SFO")
        
        to_air = driver.find_element_by_id("toAir")
        to_air.send_keys(dest)
        
        driver.find_element_by_class_name('btn-fare').click()
        
        req = requests.get(driver.current_url).text
        soup = BeautifulSoup(req)
        price = int(soup.find("th", string="Average price").find_next_sibling().text.split('$')[1])
        
        dest_dict[dest] = price
        time.sleep(5)
    
    return dest_dict


In [137]:
# Impute mean fare into missing data

def fares_transform():
    mean_fare = int(statistics.mean(list(fares_dict.values())))
    
    for fare in fares_dict:
        if fares_dict[fare] == 0:
            fares_dict[fare] = mean_fare
            
    return fares_dict


In [135]:
fares_dict = get_price_info()
fares_transform()

In [ ]:
pd.to_pickle(fares_dict, 'fares_dict.pkl')